In [16]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from threading import Thread
from PIL import Image
import numpy as np
import glob

**Definição da classe pra gerar imagens com ImageDataGenerator**

In [23]:
#Esta implementação gera novas imagens a partir das originais.
class GenerateImage(Thread):
    
    def __init__ (self, path, to, qtd):
        Thread.__init__(self)
        self.path = path
        self.to = to
        self.qtd = qtd
        
    def run(self):
        self.generate()
        
    def create_path(self,uri):
        if not os.path.isdir(uri):
            uri_ = os.path.split(uri)[0]
            self.create_path(uri_)

            os.mkdir(uri)
            
    def generate(self):
        
        self.create_path(self.to)
        
        names_images = os.listdir(self.path)

        datagenerator = ImageDataGenerator(
            rotation_range = 90,
            #width_shift_range=0.1,
            #height_shift_range=0.1,
            #rescale=1./255,
            shear_range=0.2,
            zoom_range=0.23,
            horizontal_flip=True,\
            vertical_flip=True,
            fill_mode='nearest'
        )

        for name_image in names_images:
            
            if not os.path.isfile(self.path+"/"+name_image):
                continue

            image = load_img(self.path+"/"+name_image)
            image_array = img_to_array(image)
            image_array = image_array.reshape((1,)+image_array.shape) #Reshape the image

            i = 0
            for batch in datagenerator.flow(image_array, batch_size=1, save_to_dir=self.to, save_prefix="g", save_format='jpg'):
              
              i = i+1
              if (i >= self.qtd):
                break


**redimensionar as imagens para Uma resolução desejada**

In [18]:
#redimensiona cada imagem da path para o tamanho correspondente
class CropScalingImage(Thread):
    
    def __init__ (self, path, to_path, r, with_crop, with_resize):
        Thread.__init__(self)
        self.path = path
        self.to_path = to_path
        self.r = r
        self.with_crop = with_crop
        self.with_resize = with_resize
        
    def run(self):
        self.scaling()
        
    def create_path(self,uri):
        if not os.path.isdir(uri):
            uri_ = os.path.split(uri)[0]
            self.create_path(uri_)

            os.mkdir(uri)
        
    def scaling(self):
        
        self.create_path(self.to_path)
        
        for name in os.listdir(self.path):
            
            img = Image.open(self.path+"/"+name)
            img = img.convert('RGB')
            #crop image
            
            w = img.size[0]
            h = img.size[1]
            
            if self.with_crop:
                if w > h:
                    dw = (w-h)/2.0
                    img = img.crop((dw,0,w-dw,h))
                elif h > w:
                    dh = (h-w)/2.0
                    img = img.crop((0,dh,w,h-dh))
                else:
                    dw = (w-self.r)/2.0
                    dh = (h-self.r)/2.0
                    
                    rr = (dw,dh,self.r+dw,self.r+dh)
                    img = img.crop(rr)
            
            if self.with_resize:
                #scale image
                if w > h:
                    img = img.resize((self.r,self.r), Image.ANTIALIAS)
                else:
                    img = img.resize((self.r,self.r), Image.ANTIALIAS)
                    
            if not os.path.isdir(self.to_path):
                os.mkdir(self.to_path)
                
            img.save(self.to_path+"/"+name)
            

# Executar

In [19]:
uri_path = "C:/Users/JPIRES16/Documents/ThyroidData/cell_content_classifier/cell_non_224"

**Recorte e redimensionamento**

In [20]:
#recortar apenas um lado.
for sub in os.listdir(uri_path):

    for sub_sub in os.listdir(uri_path+"/"+sub):

        crop_and_rescaling = CropScalingImage(path=uri_path+"/"+sub+"/"+sub_sub, 
                                            to_path=uri_path+"/"+sub+"/"+sub_sub,
                                            r=224,
                                            with_crop=False,
                                            with_resize=True)
        crop_and_rescaling.scaling()

**Converter imagens png para jpg**

In [21]:
def png_to_jpg(image_uri):
        png = Image.open(image_uri)
        png.load() # required for png.split()

        background = Image.new("RGB", png.size, (255, 255, 255))
        background.paste(png, mask=png.split()[3]) # 3 is the alpha channel

        base = os.path.split(image_uri)
        name = base[1].replace(".png",".jpg")

        background.save(base[0]+"/"+name, 'JPEG', quality=100)

        os.remove(image_uri)

In [24]:
for p in os.listdir(uri_to_path):
    for image in glob.glob(uri_to_path+"/"+p+"/*.png"):
        png_to_jpg(image)

**Gerar imagens**

In [24]:
uri_path = "C:/Users/JPIRES16/Documents/ThyroidData/cell_content_classifier/non_cell_p"
uri_to_path = "C:/Users/JPIRES16/Documents/ThyroidData/cell_content_classifier/non_cell_p_g"

g = GenerateImage(uri_path, uri_to_path,16)
g.generate()

______________________________________________________________

**Apagar imagens em excesso**

In [ ]:
classes = os.listdir(uri_)

for classe in classes:
    q = len(os.listdir("C:/Users/Joelp/OneDrive/Imagens/sementes/preparadas/21-04-2021/gerada/"+classe))
    qtd = qtd+q
    print(q)
    
print("Quantidade total:")
print(qtd)

In [ ]:
#
i = 1
images = os.listdir("imagens/sementes/deep_learning/milho")
for image in images:
    os.remove("imagens/sementes/deep_learning/milho/"+image)
    print(".", end="")
    if i == 1316:
        break
    i = i+1

**Converter imagens png para jpeg**

In [ ]:
list_classes = os.listdir(uri_path)

for classe in list_classes:
    images = os.listdir("sementes/deep_learning/"+classe)
    print(classe)
    for image in images:
        im = Image.open("sementes/deep_learning/"+classe+"/"+image)
        rgb_im = im.convert('RGB')
        rgb_im.save("sementes/deep_learning/"+classe+"/"+image)
        print(".", end="")

**Renomear as imagens**

In [ ]:
list_classes = os.listdir(uri_to_path)
for classe in list_classes:
    list_sementes = os.listdir(uri_to_path+"/"+classe)
    i = 0
    print("\n"+classe)
    for name in list_sementes:
        
        os.rename(uri_to_path+"/"+classe+"/"+name,uri_to_path+"/"+classe+"/"+classe+"_"+str(i)+".jpeg")
        i = i+1
        print(".",end="")

### Split Dataset

In [25]:
to = "C:/Users/JPIRES16/Documents/ThyroidData/cell_content_classifier/"
path = "C:/Users/JPIRES16/Documents/ThyroidData/cell_content_classifier/non_cell_p_g"
files = os.listdir(path)

indices = np.arange(0, len(files))
np.random.shuffle(indices)
np.random.shuffle(indices)
np.random.shuffle(indices)

#train
for i in indices[0:int(0.7*len(indices))]:
    os.rename(path+"/"+files[i],to+"/train/non/"+os.path.split(files[i])[1])

#val
for i in indices[int(0.7*len(indices)):]:
    os.rename(path+"/"+files[i],to+"/validation/non/"+os.path.split(files[i])[1])
